In [1]:
import random
from typing import List, Tuple

import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from torch.distributions import Normal
from unityagents import UnityEnvironment
import numpy as np
from reacher.src.networks import * 
from reacher.src.utils import *

In [2]:
device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
print(device)

cuda


In [3]:
file_name = './Reacher_Windows_x86_64_20/Reacher_Windows_x86_64/Reacher.exe'
worker_id = 10
base_port = 5006
env, brain_name, brain, action_size, env_info, state, state_size, n_agents = load_env(worker_id,
                                                                                                  base_port, file_name,
                                                                                                  True, True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [12]:
is_test = False
lr = 3e-4
gamma = 0.99
batch_size = 256
buffer_size = int(1e6)
alpha = .2
tau = 0.01
target_update_interval = 1
gradient_steps = 1
num_episodes= 10000 

In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [6]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  7.90150833e+00 -1.00000000e+00
  1.25147629e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -5.22214413e-01]


In [ ]:
#env.close()

In [137]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents) 
states,num_agents

(array([[ 0.00000000e+00, -4.00000000e+00,  0.00000000e+00,
          1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -4.37113883e-08,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00, -1.00000000e+01,
          0.00000000e+00,  1.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -4.37113883e-08,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00, -6.70936584e+00,
         -1.00000000e+00, -4.35711288e+00,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00,  9.06391859e-01],
        [ 0.00000000e+00, -4.00000000e+00,  0.00000000e+00,
          1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -4.37113883e-08,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00, -1.00000000e+01,
          0.00000000e+00,  1.00000000e+

In [138]:
states.shape

(20, 33)

In [139]:
for i in range(num_agents):
    print(states[i,:])

[ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -6.70936584e+00 -1.00000000e+00
 -4.35711288e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
  9.06391859e-01]
[ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  4.58861160e+00 -1.00000000e+00
 -6.553

In [10]:
state_size, action_size

(33, 4)

In [11]:
obs_dim = state_size
action_dim = action_size
obs_dim, action_dim

(33, 4)

In [13]:
policy_network = Policy_SAC(state_size=state_size, action_size=action_size,
                                 n_agents=n_agents).to(device)
policy_optimizer = optim.Adam(policy_network.parameters(), lr=lr)

# Initialize Value Network
value_network_local = Value_SAC(state_size=state_size, action_size=action_size,
                                     n_agents=n_agents).to(device)
value_network_target = Value_SAC(state_size=state_size, action_size=action_size,
                                      n_agents=n_agents).to(device)
value_optimizer = optim.Adam(value_network_local.parameters(), lr=lr)

# Initialize Q Network
q_network_1 = Q_SAC(state_size=state_size, action_size=action_size,
                         n_agents=n_agents).to(device)
q_network_2 = Q_SAC(state_size=state_size, action_size=action_size,
                         n_agents=n_agents).to(device)
q_optimizer_1 = optim.Adam(q_network_1.parameters(), lr=lr)
q_optimizer_2 = optim.Adam(q_network_2.parameters(), lr=lr)

# Initialize Replay Memory
memory = ReplayBuffer_2(action_size, buffer_size, batch_size, 0)

value_criterion = nn.MSELoss()
soft_q_criterion1 = nn.MSELoss()
soft_q_criterion2 = nn.MSELoss()

In [14]:
def reset(env, num_agents ):
    state = env.reset()[brain_name].vector_observations.reshape(num_agents, -1)
    return state

In [15]:
brain = env.brains[brain_name]
action_size = brain.vector_action_space_size
action_size

4

In [31]:
score = 0

In [16]:
states = reset(env, n_agents)
states

array([[ 0.00000000e+00, -4.00000000e+00,  0.00000000e+00,
         1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -4.37113883e-08,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.00000000e+01,
         0.00000000e+00,  1.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -4.37113883e-08,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  7.90150833e+00,
        -1.00000000e+00,  1.25147629e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -1.29508138e-01],
       [ 0.00000000e+00, -4.00000000e+00,  0.00000000e+00,
         1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -4.37113883e-08,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.00000000e+01,
         0.00000000e+00,  1.00000000e+00, -0.00000000e+

In [22]:
def act( state, episode):
    state = torch.from_numpy(state).float().to(device)
    policy_network.eval()
    with torch.no_grad():
        mean, std = policy_network(state)
    policy_network.train()

    std = std.exp()
    normal = Normal(0, 1)
    z = normal.sample().to(device)
    action = torch.tanh(mean + std * z)
    action = torch.clamp(action, -1, 1)

    return action.detach().cpu().numpy()

def interact(action, num_agents):
    action = action.reshape(num_agents, -1)
    env_info = env.step(action)[brain_name]
    next_state, reward, done = env_info.vector_observations, env_info.rewards, env_info.local_done
    return next_state.reshape(num_agents, -1), np.array(reward).reshape(num_agents, -1), np.array(done).reshape(
        num_agents, -1)

In [94]:
actions = act(states, 1)
actions.shape

(20, 4)

In [95]:
next_states, rewards, dones = interact(actions, n_agents)
next_states, rewards, dones

(array([[-1.42551422e-01, -3.99545431e+00,  1.28723145e-01,
          9.99714434e-01, -1.77307297e-02,  2.81307381e-04,
          1.60195380e-02,  1.76793113e-01,  5.63299283e-03,
          1.58210874e-01,  6.35941565e-01, -4.98391204e-02,
         -7.08859980e-01,  8.13125610e-01, -9.61333847e+00,
         -7.76081085e-01,  9.67096746e-01,  1.75640091e-01,
          3.97916213e-02, -1.79697305e-01, -3.68821830e-01,
          1.58638015e-01, -1.74829423e-01,  1.93480417e-01,
         -9.22762156e-01, -6.15897179e-02,  7.70022964e+00,
         -1.00000000e+00,  2.16943741e+00,  0.00000000e+00,
          1.00000000e+00,  0.00000000e+00, -1.29508138e-01],
        [-1.64405823e-01, -3.99474907e+00,  1.24595642e-01,
          9.99670565e-01, -2.04499140e-02,  3.14456294e-04,
          1.55082634e-02,  1.78511500e-01,  5.38304308e-03,
          1.55506194e-01,  6.24909043e-01, -5.23158200e-02,
         -7.15545952e-01,  7.99076080e-01, -9.60015488e+00,
         -7.88623810e-01,  9.65971112e-

In [96]:
#step_counter = 0
def step( states, actions, rewards, next_states, dones, step_counter):
    for i in range(states.shape[0]):
        memory.add(states[i], actions[i], rewards[i], next_states[i], dones[i])
        step_counter += 1

    if step_counter >= target_update_interval and len(memory.memory) > batch_size:
        print("memoery_ready")
        step_counter = 0
    return step_counter

In [97]:
step_counter = step(states, actions, rewards, next_states, dones, step_counter)
#step_counter

memoery_ready


In [98]:
step_counter

0

In [99]:
states = next_states
score += rewards.mean()
score

0.0

In [100]:
states, actions, rewards, next_states, dones = memory.sample(device)

In [101]:
states, actions, rewards, next_states, dones

(tensor([[-0.2106, -3.9886,  0.2180,  ...,  1.0000,  0.0000, -0.3968],
         [-0.2104, -3.9889,  0.2120,  ...,  1.0000,  0.0000, -0.8596],
         [ 0.0000, -4.0000,  0.0000,  ...,  1.0000,  0.0000,  0.5688],
         ...,
         [ 0.0300, -3.9998, -0.0233,  ...,  1.0000,  0.0000, -0.1295],
         [-0.1569, -3.9919,  0.2037,  ...,  1.0000,  0.0000,  0.9748],
         [-0.0141, -3.9997,  0.0502,  ...,  1.0000,  0.0000,  0.9748]],
        device='cuda:0'),
 tensor([[-1.0000, -1.0000, -1.0000, -1.0000],
         [ 0.9994,  0.9994,  0.9993,  0.9994],
         [ 0.1010,  0.1215,  0.1301,  0.1224],
         ...,
         [ 0.9987,  0.9989,  0.9987,  0.9988],
         [-0.3981, -0.3820, -0.3618, -0.3742],
         [ 0.3536,  0.3644,  0.3869,  0.3702]], device='cuda:0'),
 tensor([[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
        

In [102]:
predicted_q_value1 = q_network_1(states, actions)
#predicted_q_value1

In [103]:
predicted_q_value1

tensor([[-1.0386e-02],
        [-6.4869e-03],
        [ 4.9439e-03],
        [ 1.7518e-03],
        [-7.4410e-03],
        [-1.1458e-02],
        [-6.3743e-03],
        [-8.7954e-03],
        [-1.6423e-03],
        [ 4.2446e-03],
        [-9.3478e-03],
        [-7.9160e-03],
        [ 4.4126e-03],
        [ 4.6498e-05],
        [-7.3506e-03],
        [ 7.6969e-04],
        [ 2.1522e-03],
        [ 5.7314e-03],
        [ 3.7552e-03],
        [-9.4600e-03],
        [-1.0800e-02],
        [-1.0488e-02],
        [-8.9419e-03],
        [-3.8261e-03],
        [-1.0220e-02],
        [ 1.7306e-03],
        [-7.1144e-03],
        [ 2.5304e-03],
        [-9.3046e-03],
        [-1.4841e-03],
        [ 6.7085e-03],
        [-6.5830e-04],
        [ 4.4679e-03],
        [-9.0729e-03],
        [-7.9735e-03],
        [-7.1820e-03],
        [-1.0538e-02],
        [ 2.9662e-03],
        [-7.6759e-03],
        [ 4.1307e-03],
        [-1.4467e-03],
        [ 3.5375e-03],
        [ 7.0753e-03],
        [-1

In [104]:
predicted_q_value2 = q_network_2(states, actions)

In [105]:
predicted_value = value_network_local(states)

In [106]:
new_action, log_prob, epsilon, mean, log_std = policy_network.evaluate(states, device)

In [107]:
target_value = value_network_target(next_states)

In [108]:
target_q_value = rewards + (1 - dones) * gamma * target_value

In [109]:
q_value_loss1 = soft_q_criterion1(predicted_q_value1, target_q_value.detach()).mean()

In [110]:
q_value_loss2 = soft_q_criterion2(predicted_q_value2, target_q_value.detach()).mean()

In [111]:
q_value_loss1, q_value_loss2

(tensor(0.0006, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(0.0009, device='cuda:0', grad_fn=<MeanBackward0>))

In [114]:
from torch.cuda.amp.grad_scaler import GradScaler
scaler = GradScaler()
gradient_clip = 5

In [115]:
q_optimizer_1.zero_grad()
scaler.scale(q_value_loss1).backward()
scaler.unscale_(q_optimizer_1)
nn.utils.clip_grad_norm_(q_network_1.parameters(), gradient_clip)  # clip gradient
scaler.step(q_optimizer_1)
scaler.update()

In [116]:
q_optimizer_2.zero_grad()
scaler.scale(q_value_loss2).backward()
scaler.unscale_(q_optimizer_2)
nn.utils.clip_grad_norm_(q_network_2.parameters(), gradient_clip)  # clip gradient
scaler.step(q_optimizer_2)
scaler.update()

In [117]:
predicted_new_q_value = torch.min(q_network_1(states, new_action), q_network_2(states, new_action))

In [126]:
target_value_func = predicted_new_q_value - log_prob.sum(dim=1, keepdim=True)*0.2

In [125]:
log_prob.sum(dim=1, keepdim=True)*0.2

tensor([[4.2228],
        [4.2546],
        [4.2229],
        [4.3533],
        [4.3097],
        [4.3042],
        [4.2280],
        [4.3286],
        [4.2799],
        [4.1519],
        [4.2735],
        [4.2837],
        [4.4006],
        [4.2766],
        [4.2944],
        [4.2959],
        [4.3047],
        [4.3113],
        [4.3531],
        [4.2926],
        [4.2521],
        [4.2770],
        [4.2919],
        [4.2621],
        [4.2908],
        [4.3913],
        [4.3553],
        [4.3573],
        [4.2353],
        [4.2793],
        [4.2405],
        [4.2143],
        [4.3751],
        [4.2867],
        [4.2619],
        [4.1758],
        [4.2742],
        [4.2471],
        [4.1218],
        [4.3452],
        [4.2821],
        [4.2364],
        [4.2422],
        [4.2896],
        [4.1043],
        [4.2557],
        [4.3338],
        [4.2754],
        [4.2447],
        [4.3202],
        [4.2791],
        [4.2982],
        [4.3157],
        [4.3206],
        [4.2131],
        [4

In [120]:
predicted_value

tensor([[-6.1774e-03],
        [-2.7221e-03],
        [-7.7975e-04],
        [-5.7266e-03],
        [-1.0909e-02],
        [-8.6740e-03],
        [-2.2690e-03],
        [-9.9444e-03],
        [-1.9079e-04],
        [ 4.7557e-04],
        [-3.7992e-03],
        [-1.2914e-02],
        [-3.7348e-03],
        [ 1.5046e-03],
        [-1.3753e-03],
        [ 2.5480e-05],
        [-4.5223e-03],
        [-1.2105e-03],
        [-3.9252e-04],
        [-3.7619e-03],
        [-1.0096e-02],
        [-6.4654e-03],
        [-3.5697e-03],
        [-7.0632e-04],
        [-1.1065e-02],
        [-4.6149e-03],
        [-1.2384e-02],
        [-1.4150e-03],
        [-5.5659e-03],
        [ 4.4706e-04],
        [ 6.9862e-04],
        [-1.4836e-03],
        [-2.4044e-03],
        [-9.0396e-03],
        [-1.4792e-03],
        [-3.5642e-03],
        [-1.2034e-03],
        [-2.2214e-03],
        [-3.2350e-03],
        [-3.4250e-04],
        [ 2.9516e-03],
        [ 4.9202e-04],
        [-1.5738e-03],
        [-1

In [121]:
target_value_func

tensor([[-5.0888, -5.7483, -5.4013, -4.8215],
        [-5.3952, -5.2864, -5.4495, -5.0665],
        [-5.1023, -5.4634, -5.3264, -5.1266],
        ...,
        [-5.3252, -5.1898, -5.5158, -5.0336],
        [-5.2379, -5.2138, -5.9535, -5.1071],
        [-5.1955, -5.2905, -5.5726, -5.1305]], device='cuda:0',
       grad_fn=<SubBackward0>)

In [122]:
predicted_new_q_value

tensor([[0.0135],
        [0.0189],
        [0.0240],
        [0.0229],
        [0.0151],
        [0.0145],
        [0.0210],
        [0.0160],
        [0.0239],
        [0.0245],
        [0.0166],
        [0.0130],
        [0.0225],
        [0.0242],
        [0.0175],
        [0.0236],
        [0.0210],
        [0.0251],
        [0.0245],
        [0.0154],
        [0.0158],
        [0.0154],
        [0.0153],
        [0.0186],
        [0.0154],
        [0.0208],
        [0.0145],
        [0.0250],
        [0.0150],
        [0.0181],
        [0.0270],
        [0.0182],
        [0.0229],
        [0.0160],
        [0.0177],
        [0.0192],
        [0.0154],
        [0.0231],
        [0.0181],
        [0.0246],
        [0.0169],
        [0.0264],
        [0.0236],
        [0.0156],
        [0.0170],
        [0.0203],
        [0.0199],
        [0.0257],
        [0.0263],
        [0.0266],
        [0.0232],
        [0.0243],
        [0.0243],
        [0.0231],
        [0.0177],
        [0

In [127]:
value_loss = value_criterion(predicted_value, target_value_func.detach()).mean()

In [128]:
value_loss

tensor(18.0631, device='cuda:0', grad_fn=<MeanBackward0>)

In [129]:
value_optimizer.zero_grad()
scaler.scale(value_loss).backward()
scaler.unscale_(value_optimizer)
nn.utils.clip_grad_norm_(value_network_local.parameters(), gradient_clip)  # clip gradient
scaler.step(value_optimizer)
scaler.update()

In [130]:
policy_loss = (log_prob - predicted_new_q_value).mean()

In [131]:
log_prob - predicted_new_q_value

tensor([[5.0888, 5.7483, 5.4013, 4.8215],
        [5.3952, 5.2864, 5.4495, 5.0665],
        [5.1023, 5.4634, 5.3264, 5.1266],
        ...,
        [5.3252, 5.1898, 5.5158, 5.0336],
        [5.2379, 5.2138, 5.9535, 5.1071],
        [5.1955, 5.2905, 5.5726, 5.1305]], device='cuda:0',
       grad_fn=<SubBackward0>)

In [132]:
predicted_new_q_value

tensor([[0.0135],
        [0.0189],
        [0.0240],
        [0.0229],
        [0.0151],
        [0.0145],
        [0.0210],
        [0.0160],
        [0.0239],
        [0.0245],
        [0.0166],
        [0.0130],
        [0.0225],
        [0.0242],
        [0.0175],
        [0.0236],
        [0.0210],
        [0.0251],
        [0.0245],
        [0.0154],
        [0.0158],
        [0.0154],
        [0.0153],
        [0.0186],
        [0.0154],
        [0.0208],
        [0.0145],
        [0.0250],
        [0.0150],
        [0.0181],
        [0.0270],
        [0.0182],
        [0.0229],
        [0.0160],
        [0.0177],
        [0.0192],
        [0.0154],
        [0.0231],
        [0.0181],
        [0.0246],
        [0.0169],
        [0.0264],
        [0.0236],
        [0.0156],
        [0.0170],
        [0.0203],
        [0.0199],
        [0.0257],
        [0.0263],
        [0.0266],
        [0.0232],
        [0.0243],
        [0.0243],
        [0.0231],
        [0.0177],
        [0

In [ ]:
state=torch.tensor(states).float().to(device)

In [ ]:
actor_local.eval()
with torch.no_grad():
    action = actor_local(state).cpu().data.numpy()
actor_local.train()

In [ ]:
action += noise.sample()
action = np.clip(action, -1.0, 1.0)
action

In [ ]:
env_info = env.step(action)[brain_name]  # send the action to the environment
next_states = env_info.vector_observations  # get the next state
rewards = env_info.rewards  # get the reward
dones = env_info.local_done
next_states, rewards,dones

In [ ]:
memory.add(states, action, rewards, next_states, dones)

In [ ]:
memory.__len__()

In [ ]:
#critic_local(states, action)

In [ ]:
actor = Actor(obs_dim, action_dim).to(device)
critic = Critic(obs_dim).to(device)

In [ ]:
actor_optimizer = optim.Adam(actor.parameters(), lr=1e-4)
critic_optimizer = optim.Adam(critic.parameters(), lr=1e-3)

In [ ]:
state = torch.from_numpy(states).float().to(device)
state

In [ ]:
action, dist = actor(state)
action, dist

In [ ]:
selected_action = dist.mean if is_test else action
selected_action

In [ ]:
if not is_test:
    log_prob = dist.log_prob(selected_action).sum(dim=-1)
    transition = [state, log_prob]
transition

In [ ]:
selected_action = selected_action.clamp(-2.0, 2.0).cpu().detach().numpy()
selected_action

In [ ]:
next_state, reward, done, _ = env.step(selected_action)
next_state, reward, done

In [ ]:
if not is_test:
    transition.extend([next_state, reward, done])  
transition

In [ ]:
state, log_prob, next_state, reward, done = transition
state, log_prob, next_state, reward, done

In [ ]:
mask = 1 - done
mask

In [ ]:
next_state = torch.FloatTensor(next_state).to(device)
next_state

In [ ]:
pred_value = critic(state)

In [ ]:
pred_value

In [ ]:
targ_value = reward + gamma * critic(next_state) * mask
targ_value

In [ ]:
value_loss = F.smooth_l1_loss(pred_value, targ_value.detach())
value_loss

In [ ]:
# update value
critic_optimizer.zero_grad()
value_loss.backward()
critic_optimizer.step()

In [ ]:
# advantage = Q_t - V(s_t)
advantage = (targ_value - pred_value).detach()
advantage

In [ ]:
policy_loss = -advantage * log_prob
policy_loss

In [ ]:
policy_loss += entropy_weight * -log_prob
policy_loss

In [ ]:
# update policy
actor_optimizer.zero_grad()
policy_loss.backward()
actor_optimizer.step()

In [ ]:
policy_loss.item(), value_loss.item()